In [183]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture
import sklearn
import numpy as np
import pandas as pd
import conversion_df

# Récuperation du json et conversion en quelque chose d'utilisable

# Conversion en base de données panda et nettoyage

## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

## Création de dummys


In [189]:
df = conversion_df.data_frame_pandas('donnees_citroen.json')

In [190]:
df.dtypes


modele                    object
carburant                 object
prix                       int64
kilometrage              float64
garantie_kilometrage      object
boite_de_vitesse          object
transmission             float64
couleur                   object
garantie                  object
date_mise_circulation      int64
puissance                float64
silhouette                object
nb_places                float64
utilisation_prec          object
puissance_fiscale        float64
critair                  float64
ptac                     float64
nb_portes                float64
dtype: object

In [191]:
df.ptac

0      1835.0
1      2300.0
2      1735.0
3      1540.0
4      1600.0
        ...  
595    1795.0
596    1610.0
597    1610.0
598    1610.0
599    1610.0
Name: ptac, Length: 600, dtype: float64

In [192]:
df.convert_dtypes().dtypes

modele                    string
carburant                 string
prix                       Int64
kilometrage              Float64
garantie_kilometrage      string
boite_de_vitesse          string
transmission               Int64
couleur                   string
garantie                  string
date_mise_circulation      Int64
puissance                Float64
silhouette                string
nb_places                Float64
utilisation_prec          string
puissance_fiscale        Float64
critair                  Float64
ptac                     Float64
nb_portes                Float64
dtype: object

## Passage sous numpy + ML

In [193]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
import conversion_df

reload(conversion_df)

<module 'conversion_df' from 'c:\\Users\\mathi\\Documents\\Cours\\M2\\Machine Learning\\VERON_NEPVEUX_PROJECT\\veron_nepveux_project\\conversion_df.py'>

In [194]:
X, y = conversion_df.data_frame_modele('donnees_citroen.json')

In [195]:
X

array([[20629.,  2021.,   110., ...,     0.,     0.,     0.],
       [ 5104.,  2022.,   181., ...,     0.,     0.,     0.],
       [ 8900.,  2021.,   131., ...,     0.,     0.,     0.],
       ...,
       [   30.,  2022.,   110., ...,     0.,     0.,     0.],
       [   30.,  2022.,   110., ...,     0.,     0.,     0.],
       [   30.,  2022.,   110., ...,     0.,     0.,     0.]])

In [196]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [197]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)
en_df.head()
        

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.297e+08, tolerance: 1.955e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.382e+08, tolerance: 2.229e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023535,0.005659,0.000994,0.000631,0.015625,0.01,"{'alpha': 0.015625, 'l1_ratio': 0.01}",0.894038,0.914747,0.892849,0.918680,0.898777,0.903818,0.010786,16
1,0.015957,0.001089,0.000399,0.000488,0.015625,0.25,"{'alpha': 0.015625, 'l1_ratio': 0.25}",0.899720,0.915369,0.896189,0.921021,0.903125,0.907085,0.009503,15
2,0.023568,0.004683,0.000567,0.000464,0.015625,0.5,"{'alpha': 0.015625, 'l1_ratio': 0.5}",0.907773,0.915394,0.899678,0.923603,0.907909,0.910871,0.008077,13
3,0.036303,0.009946,0.000598,0.000489,0.015625,0.75,"{'alpha': 0.015625, 'l1_ratio': 0.75}",0.920262,0.913786,0.902274,0.926609,0.912487,0.915083,0.008150,11
4,0.105939,0.006449,0.000705,0.000379,0.015625,1,"{'alpha': 0.015625, 'l1_ratio': 1}",0.944220,0.908411,0.893623,0.934955,0.921566,0.920555,0.018117,9


In [198]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 2, 'l1_ratio': 1}, 0.922197217213192)

In [199]:
from sklearn.neighbors import KNeighborsRegressor

KNN

In [200]:
knr = KNeighborsRegressor()
knr_gs = GridSearchCV(
    knr,
    {
        "n_neighbors": range(5, 15),
        "weights": ("uniform", "distance"),
    }
)

In [201]:
knr_gs.fit(X_tr, y_tr)
knr_df = pd.DataFrame(knr_gs.cv_results_)
knr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000447,0.000550,0.011718,0.005628,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.394768,0.251342,0.025068,0.161833,0.123955,0.191393,0.124933,12
1,0.000399,0.000489,0.009768,0.001151,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.444726,0.333097,0.031749,0.222572,0.137956,0.234020,0.144559,9
2,0.000595,0.000486,0.007971,0.000880,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.355400,0.249664,0.083008,0.191035,0.129295,0.201680,0.095256,11
3,0.000809,0.000406,0.008561,0.001508,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.425699,0.329144,0.083163,0.249399,0.153053,0.248091,0.121942,1
4,0.000399,0.000489,0.006383,0.001016,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.312301,0.217796,0.060646,0.148042,0.142852,0.176328,0.084275,13


In [202]:
knr_gs.best_params_, knr_gs.best_score_

({'n_neighbors': 6, 'weights': 'distance'}, 0.24809144009222037)

In [203]:
from sklearn.ensemble import RandomForestRegressor

In [204]:
rfr = RandomForestRegressor()
rfr_gs = GridSearchCV(
    rfr,
    {   
        "n_estimators": (8 , 16, 32, 64, 128, 256),
    }
)

In [205]:
rfr_gs.fit(X_tr, y_tr)
rfr_df = pd.DataFrame(rfr_gs.cv_results_)
rfr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.027923,0.000891,0.001197,0.000399,8,{'n_estimators': 8},0.910364,0.846120,0.871130,0.826891,0.856757,0.862252,0.028051,6
1,0.053058,0.002926,0.001796,0.000400,16,{'n_estimators': 16},0.914589,0.864101,0.867215,0.854030,0.868530,0.873693,0.021070,5
2,0.119287,0.011323,0.003591,0.000798,32,{'n_estimators': 32},0.928112,0.876500,0.870193,0.858463,0.887544,0.884162,0.023905,4
3,0.200071,0.018566,0.004781,0.000396,64,{'n_estimators': 64},0.927947,0.881107,0.880629,0.859547,0.874914,0.884829,0.022927,3
4,0.362430,0.010195,0.008578,0.001196,128,{'n_estimators': 128},0.924498,0.883875,0.872230,0.866710,0.887871,0.887037,0.020231,2


In [206]:
rfr_gs.best_params_, rfr_gs.best_score_

({'n_estimators': 256}, 0.8876187424461225)

In [207]:
from sklearn.svm import SVR

In [208]:
svr = SVR()
svr_gs = GridSearchCV(
    svr,
    {
        "C": (0.1, 1.0, 10),
        "epsilon": (0.1, 1.0, 10),
    }
)
svr_gs.fit(X_tr, y_tr)

svr_df = pd.DataFrame(svr_gs.cv_results_)
svr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009574,0.001196,0.004787,0.000400,0.1,0.1,"{'C': 0.1, 'epsilon': 0.1}",-0.052352,-0.041738,-0.022034,-0.036633,-0.149735,-0.060498,0.045674,9
1,0.008577,0.000489,0.004588,0.000488,0.1,1.0,"{'C': 0.1, 'epsilon': 1.0}",-0.052352,-0.041717,-0.022034,-0.036631,-0.149735,-0.060494,0.045676,8
2,0.008984,0.000631,0.004387,0.000491,0.1,10,"{'C': 0.1, 'epsilon': 10}",-0.052352,-0.041246,-0.022034,-0.036142,-0.149735,-0.060302,0.045766,7
3,0.009360,0.000790,0.004602,0.000500,1.0,0.1,"{'C': 1.0, 'epsilon': 0.1}",-0.050967,-0.040301,-0.021590,-0.035221,-0.146704,-0.058957,0.044880,5
4,0.008783,0.000403,0.004781,0.000396,1.0,1.0,"{'C': 1.0, 'epsilon': 1.0}",-0.050967,-0.040310,-0.021590,-0.035221,-0.146704,-0.058958,0.044880,6


In [209]:
svr_gs.best_params_, svr_gs.best_score_

({'C': 10, 'epsilon': 0.1}, -0.04565444631037137)

In [210]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [211]:
pl = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("support_vecteurs", SVR()),
    ]
)

In [212]:
pl_gs = GridSearchCV(
    pl,
    {
        "support_vecteurs__C": (0.1, 1.0, 10),
        "support_vecteurs__epsilon": (0.1, 1.0, 10),
    }
)

In [213]:
pl_gs.fit(X_tr, y_tr)

pl_df = pd.DataFrame(pl_gs.cv_results_)
pl_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_support_vecteurs__C,param_support_vecteurs__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009781,0.000987,0.004787,0.000400,0.1,0.1,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.052153,-0.041614,-0.021666,-0.036429,-0.149600,-0.060292,0.045722,9
1,0.009756,0.000392,0.004605,0.000503,0.1,1.0,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.052153,-0.041591,-0.021666,-0.036399,-0.149600,-0.060282,0.045727,8
2,0.008949,0.000069,0.004187,0.000400,0.1,10,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.052153,-0.041143,-0.021666,-0.035908,-0.149600,-0.060094,0.045815,7
3,0.009577,0.000492,0.004578,0.000501,1.0,0.1,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.048983,-0.039056,-0.017911,-0.033178,-0.145352,-0.056896,0.045360,5
4,0.009182,0.000396,0.003983,0.000013,1.0,1.0,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.048983,-0.039064,-0.017911,-0.033178,-0.145352,-0.056898,0.045360,6


In [214]:
pl_gs.best_params_, pl_gs.best_score_

({'support_vecteurs__C': 10, 'support_vecteurs__epsilon': 10},
 -0.02352431249443543)

In [215]:
from sklearn.neural_network import MLPRegressor
import numpy as np

In [216]:
pln = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neurones", MLPRegressor()),
    ]
)

In [217]:
pln_gs = GridSearchCV(
    pln,
    {
        "neurones__alpha": 10.0 ** -np.arange(1, 7),
        'neurones__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
    }
)

In [218]:
pln_gs.fit(X_tr, y_tr)

pln_df = pd.DataFrame(pln_gs.cv_results_)
pln_df.head()

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_neurones__alpha,param_neurones__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.273378,0.007361,0.001017,0.000017,0.1,"(25,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-5.432010,-7.683767,-9.449958,-9.403328,-8.439998,-8.081812,1.478270,20
1,0.514626,0.005831,0.000798,0.000399,0.1,"(50,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-5.390329,-7.620881,-9.390750,-9.321108,-8.408702,-8.026354,1.469502,17
2,0.605408,0.037933,0.000604,0.000493,0.1,"(100,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-5.316770,-7.472490,-9.274609,-9.198453,-8.287049,-7.909874,1.454695,10
3,0.195483,0.014089,0.000997,0.000002,0.1,"(20, 20)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-4.940322,-6.911076,-8.914810,-8.392282,-7.580262,-7.347750,1.384715,3
4,0.278756,0.016807,0.000811,0.000406,0.01,"(25,)","{'neurones__alpha': 0.01, 'neurones__hidden_la...",-5.446838,-7.674505,-9.460567,-9.398430,-8.457199,-8.087508,1.475411,24


In [219]:
pln_gs.best_params_

pln_gs.best_score_

-7.154981346536397

In [220]:
pln_gs.best_params_

{'neurones__alpha': 0.0001, 'neurones__hidden_layer_sizes': (20, 20)}

In [221]:
essai = en_gs.best_estimator_

In [222]:
prix_prédits = essai.predict(X)

In [223]:
prix_reels = y

In [224]:
prix_reels,prix_prédits

(array([22990, 42790, 23990, 17290, 17490, 26980, 37390, 17990, 18490,
        13790, 12280, 29790, 33790, 11480, 27290, 14990, 10990, 13990,
        31990, 23990, 10990, 23990, 19990, 11990, 24990, 23490, 34990,
        22990,  9790, 13990, 15990, 13790, 26790, 19390, 26790, 13990,
        24990, 26990, 27990, 22890, 14990, 21990, 26990, 15790, 15790,
        15790, 14990, 27790, 10990, 26990, 15490, 27990,  7490, 21990,
         9990, 18990, 11990, 18890, 19990, 19890, 13990, 22990, 13990,
        27990, 25990, 15980, 27490, 45990, 27490, 22990, 24990, 16990,
        15990, 20990, 26990, 26990, 43990, 14490, 24990, 19990, 18990,
        16490, 17990, 20990, 23990, 19490, 31990, 20490, 18990, 19790,
        18990,  9790, 33990, 17490, 16990, 36990, 13990, 27490, 13990,
        32990, 34990, 26890, 11990, 39000, 27900, 11500, 26900, 30900,
        31900, 29500, 49000, 18900, 18800, 17900, 22400, 25900, 16500,
        30500, 28500, 27900, 22500, 36500, 44990, 23900, 18900, 40900,
      

In [225]:
len(prix_prédits)

600

In [226]:
len(prix_reels)

600

In [227]:
prix_prédits

array([22247.82400014, 42389.00157165, 24137.29676851, 16054.71038285,
       16203.88541643, 27347.74352942, 34036.34595786, 19667.2942481 ,
       18285.93623114, 16436.44309243, 11748.05205629, 28209.90628191,
       33679.22328338, 12953.52596866, 27307.33659483, 15467.69390459,
       10202.89143117, 16317.07925927, 34680.62399175, 24780.75648061,
        9003.31693875, 25115.35741777, 20998.20166186, 17080.12491887,
       28045.40909712, 24450.86497246, 38046.50910666, 24090.93183509,
       11329.25565251, 11328.85308894, 16797.18478644, 12473.25809404,
       26762.0328209 , 19437.4136555 , 27475.39877615, 14128.64839788,
       26909.2462293 , 30449.30218577, 30868.06472798, 22546.23147778,
       16379.06576377, 21753.76353597, 27537.23271965, 17039.52425778,
       16609.4699212 , 16621.260065  , 16210.65242241, 28934.01997618,
       10644.20896283, 28894.73831518, 16450.87938158, 27061.65461832,
        6324.33250487, 23438.94770092, 10991.80648581, 20719.96829615,
      

In [228]:
prix_reels

array([22990, 42790, 23990, 17290, 17490, 26980, 37390, 17990, 18490,
       13790, 12280, 29790, 33790, 11480, 27290, 14990, 10990, 13990,
       31990, 23990, 10990, 23990, 19990, 11990, 24990, 23490, 34990,
       22990,  9790, 13990, 15990, 13790, 26790, 19390, 26790, 13990,
       24990, 26990, 27990, 22890, 14990, 21990, 26990, 15790, 15790,
       15790, 14990, 27790, 10990, 26990, 15490, 27990,  7490, 21990,
        9990, 18990, 11990, 18890, 19990, 19890, 13990, 22990, 13990,
       27990, 25990, 15980, 27490, 45990, 27490, 22990, 24990, 16990,
       15990, 20990, 26990, 26990, 43990, 14490, 24990, 19990, 18990,
       16490, 17990, 20990, 23990, 19490, 31990, 20490, 18990, 19790,
       18990,  9790, 33990, 17490, 16990, 36990, 13990, 27490, 13990,
       32990, 34990, 26890, 11990, 39000, 27900, 11500, 26900, 30900,
       31900, 29500, 49000, 18900, 18800, 17900, 22400, 25900, 16500,
       30500, 28500, 27900, 22500, 36500, 44990, 23900, 18900, 40900,
       26900, 21000,

Objectif : maximiser le surplus de l'acheteur, i.e, si l'acheteur à un budget R, et n contraintes, quel véhicule maximiserait son surplus


In [229]:
data_finale = conversion_df.data_frame_dummies(df)

In [230]:
data_finale['prix_predits'] = prix_prédits
data_finale['surplus_conso'] = data_finale['prix'] - data_finale['prix_predits']

In [231]:
data_finale

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,critair,ptac,nb_portes,BERLINGO,...,SPOTICAR ESSENTIAL,SPOTICAR PREMIUM,Ex-Auto-école,Ex-Import,Ex-Loueur,Ex-Particulier,Ex-Société,Véhicule de direction,prix_predits,surplus_conso
0,22990,20629.0,2021,110.0,5.0,6.0,2.0,1835.0,5.0,0,...,0,1,0,1,0,0,0,0,22247.824000,742.176000
1,42790,5104.0,2022,181.0,5.0,10.0,1.0,2300.0,5.0,0,...,0,1,0,0,1,0,0,0,42389.001572,400.998428
2,23990,8900.0,2021,131.0,5.0,7.0,1.0,1735.0,5.0,0,...,0,1,0,1,0,0,0,0,24137.296769,-147.296769
3,17290,1305.0,2021,83.0,5.0,4.0,1.0,1540.0,5.0,0,...,0,1,0,1,0,0,0,0,16054.710383,1235.289617
4,17490,37377.0,2019,110.0,5.0,5.0,1.0,1600.0,5.0,0,...,0,1,0,1,0,0,0,0,16203.885416,1286.114584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,17579,10598.0,2018,110.0,5.0,5.0,1.0,1795.0,5.0,0,...,0,1,0,0,0,1,0,0,17632.125078,-53.125078
596,22689,30.0,2022,110.0,5.0,6.0,1.0,1610.0,5.0,0,...,0,1,0,1,0,0,0,0,22204.125721,484.874279
597,22689,30.0,2022,110.0,5.0,6.0,1.0,1610.0,5.0,0,...,0,1,0,1,0,0,0,0,22204.125721,484.874279
598,22689,30.0,2022,110.0,5.0,6.0,1.0,1610.0,5.0,0,...,0,1,0,1,0,0,0,0,22741.548411,-52.548411
